In [1]:
import magma as m
from mantle import FullAdder

## Add2 Circuit

Now let's build a 2-bit adder using `FullAdder`. 
We'll use a simple ripple carry adder design by connecting the carry out of one full adder
to the carry in of the next full adder.
The resulting adder will accept as input a carry in,
and generate a final carry out. Here's a `logisim` diagram of the circuit we will construct:

<img src="logisim/adder.png" width="500"/>

In [2]:
class Add2(m.Circuit):
    io = m.IO(I0=m.In(m.UInt[2]), I1=m.In(m.UInt[2]), CIN=m.In(m.Bit),
              O=m.Out(m.UInt[2]), COUT=m.Out(m.Bit))
    n = len(io.I0)
    O = []
    COUT = io.CIN
    for i in range(n):
        fulladder = FullAdder()
        Oi, COUT = fulladder(io.I0[i], io.I1[i], COUT)
        O.append(Oi)
    io.O @= m.uint(O)
    io.COUT @= COUT

Although we are making an 2-bit adder,
we do this using a for loop that can be generalized to construct an n-bit adder.
Each time through the for loop we create an instance of a full adder 
by calling `FullAdder()`. 
Recall that circuits are python classes,
so that calling a class returns an instance of that class.

Note how we wire up the full adders.
Calling an circuit instance has the effect of wiring
up the arguments to the inputs of the circuit.
That is,
```
O, COUT = fulladder(I0, I1, CIN)
```
is equivalent to
```
m.wire(IO, fulladder.I0)
m.wire(I1, fulladder.I1)
m.wire(CIN, fulladder.CIN)
O = fulladder.O
COUT = fulladder.COUT
```
The outputs of the circuit are returned.

Inside this loop we append single bit outputs from the full adders
to the Python list `O`. 
We also set the `CIN` of the next full adder to the `COUT` of the previous instance.

Finally, we then convert the list `O` to a `UInt(n)`. 
In addition to `Bits(n)`,
`Magma` also has built in types `UInt(n)` and `SInt(n)` 
to represent unsigned and signed ints.
`Magma` also has type conversion functions `bits`, `uint`, and `sint` to convert
between different types. 
In this example, `m.uint(C)` converts the list of bits to a `UInt(len(C))`.

## Add Generator

One question you may be asking yourself, is how can this code be generalized to produce an n-bit adder. We do this by creating an add *Generator*.
A `Generator` is a Python class that defines a static `generate` method which takes parameters and returns a circuit class.
Calling the generator with different parameter values will create and instantiate different circuits.
The power of `magma` results from being to use all the features of Python
to create powerful hardware generators.

Here is the code:

In [3]:
class Add(m.Generator):
    @staticmethod
    def generate(width: int):
        T = m.UInt[width]
        
        class _Add(m.Circuit):
            name = f'Add{width}'
            
            io = m.IO(I0=m.In(T), I1=m.In(T), CIN=m.In(m.Bit),
                      O=m.Out(T), COUT=m.Out(m.Bit))
            
            O = []
            COUT = io.CIN
            for i in range(width):
                fulladder = FullAdder()
                Oi, COUT = fulladder(io.I0[i], io.I1[i], COUT)
                O.append(Oi)
                
            io.O @= m.uint(O)
            io.COUT @= COUT
        return _Add

def add(i0, i1, cin):
    """
    We define a convenience function that instantiates the
    add generator for us based on the width of the inputs.
    """
    if len(i0) != len(i1):
        raise TypeError("add arguments must have same length")
    if not isinstance(cin, m.Bit):
        raise TypeError("add cin must be a Bit")
    if (not isinstance(i0, m.Uint[width]) and 
        not isinstance(i1, m.UInt[width])):
            raise TypeError("add expects UInt inputs")
    return Add(len(i0))(i0, i1, cin)

To generate a `Circuit` from a `Generator`, we can directly call the `generate` static method.

In [4]:
from fault import PythonTester

Add2 = Add.generate(2)
add2 = PythonTester(Add2)

print(add2(1,2,0)[0] == 3)
assert add2(1, 2, 0) == (3, 0), "Failed"
print("Success!")

Bit(True)
Success!


In [5]:
m.compile("build/Add2", Add2, inline=True)
%cat build/Add2.v

module full_adder (
    input A,
    input B,
    input C,
    output O0,
    output O1
);
assign O0 = (A ^ B) ^ C;
assign O1 = ((A & B) | (B & C)) | (C & A);
endmodule

module FullAdder (
    input I0,
    input I1,
    input CIN,
    output O,
    output COUT
);
full_adder full_adder_inst0 (
    .A(I0),
    .B(I1),
    .C(CIN),
    .O0(O),
    .O1(COUT)
);
endmodule

module Add2 (
    input [1:0] I0,
    input [1:0] I1,
    input CIN,
    output [1:0] O,
    output COUT
);
wire FullAdder_inst0_O;
wire FullAdder_inst0_COUT;
wire FullAdder_inst1_O;
FullAdder FullAdder_inst0 (
    .I0(I0[0]),
    .I1(I1[0]),
    .CIN(CIN),
    .O(FullAdder_inst0_O),
    .COUT(FullAdder_inst0_COUT)
);
FullAdder FullAdder_inst1 (
    .I0(I0[1]),
    .I1(I1[1]),
    .CIN(FullAdder_inst0_COUT),
    .O(FullAdder_inst1_O),
    .COUT(COUT)
);
assign O = {FullAdder_inst1_O,FullAdder_inst0_O};
endmodule



In [6]:
m.compile("build/Add2", FullAdder, output="coreir")
%cat build/Add2.json

{"top":"global.FullAdder",
"namespaces":{
  "global":{
    "modules":{
      "Add2":{
        "type":["Record",[
          ["I0",["Array",2,"BitIn"]],
          ["I1",["Array",2,"BitIn"]],
          ["CIN","BitIn"],
          ["O",["Array",2,"Bit"]],
          ["COUT","Bit"]
        ]],
        "instances":{
          "FullAdder_inst0":{
            "modref":"global.FullAdder"
          },
          "FullAdder_inst1":{
            "modref":"global.FullAdder"
          }
        },
        "connections":[
          ["self.CIN","FullAdder_inst0.CIN"],
          ["FullAdder_inst1.CIN","FullAdder_inst0.COUT"],
          ["self.I0.0","FullAdder_inst0.I0"],
          ["self.I1.0","FullAdder_inst0.I1"],
          ["self.O.0","FullAdder_inst0.O"],
          ["self.COUT","FullAdder_inst1.COUT"],
          ["self.I0.1","FullAdder_inst1.I0"],
          ["self.I1.1","FullAdder_inst1.I1"],
          ["self.O.1","FullAdder_inst1.O"]
        ]
      },
      "FullAdder":{
        "type":["Record",[
 

In [7]:
!coreir -i build/Add2.json -p instancecount

An instance count of all the primitives
full_adder | instances in current | instances in children | 
  corebit_and | 3 | 0
  corebit_or | 2 | 0
  corebit_xor | 2 | 0

FullAdder | instances in current | instances in children | 
  corebit_and | 0 | 3
  corebit_or | 0 | 2
  corebit_xor | 0 | 2

Add2 | instances in current | instances in children | 
  corebit_and | 0 | 6
  corebit_or | 0 | 4
  corebit_xor | 0 | 4

{"top":"global.FullAdder",
"namespaces":{
  "global":{
    "modules":{
      "Add2":{
        "type":["Record",[
          ["I0",["Array",2,"BitIn"]],
          ["I1",["Array",2,"BitIn"]],
          ["CIN","BitIn"],
          ["O",["Array",2,"Bit"]],
          ["COUT","Bit"]
        ]],
        "instances":{
          "FullAdder_inst0":{
            "modref":"global.FullAdder"
          },
          "FullAdder_inst1":{
            "modref":"global.FullAdder"
          }
        },
        "connections":[
          ["self.CIN","FullAdder_inst0.CIN"],
          ["FullAdder_inst1.CI